In [1]:
import os
import sys
import pickle
from time import time
import pandas as pd
from joblib import Parallel, delayed  
import multiprocessing

# Local Imports
path = str(os.path.expanduser('~')) + '/git/predictEPL/config'
sys.path.append(path)
import paths

sys.path.append(paths.UTILS)
import emolex
import useful_methods

In [2]:
# *******************************************************
# *******************************************************
# which week
WEEK_NUM = 23

# Limitations
TIME_LIMIT = 60
RETWEET_STATUS = False
FILTER_STATUS = True
START_TIME = 1
END_TIME = 60

In [4]:
useful_methods.DropNanGames(dfGameInfos)

,GW,away_team,date,home_team,score_ft_away,score_ft_home,score_ht_away,score_ht_home,time
0,5,Chelsea,Sat. 12 Sep.,Everton,1,3,1,2,11:45
1,5,Bournemouth,Sat. 12 Sep.,Norwich,1,3,0,1,14:00
2,5,Swansea,Sat. 12 Sep.,Watford,0,1,0,0,14:00
3,5,Southampton,Sat. 12 Sep.,WestBromwich,0,0,0,0,14:00
4,5,Stoke,Sat. 12 Sep.,Arsenal,0,2,0,1,14:00
5,5,City,Sat. 12 Sep.,Crystal,1,0,0,0,14:00
6,5,Liverpool,Sat. 12 Sep.,United,1,3,0,0,16:30
7,5,Tottenham,Sun. 13 Sep.,Sunderland,1,0,0,0,12:30
8,5,Villa,Sun. 13 Sep.,Leicester,2,3,1,0,15:00
9,5,Newcastle,Mon. 14 Sep.,WestHam,0,2,0,1,19:00


In [3]:
# *******************************************************
# *******************************************************

# Game Infos
os.chdir(paths.READ_PATH_GAME_INFO)
dfGameInfos = useful_methods.csv_dic_df('game_infos.csv')
dfGameInfos = useful_methods.DropNanGames(dfGameInfos)

# Convert number strings to integers
dfGameInfos['GW'] = [int(GW) for GW in dfGameInfos['GW']]
dfGameInfos['score_ht_home'] = [int(number) for number in dfGameInfos['score_ht_home']]
dfGameInfos['score_ht_away'] = [int(number) for number in dfGameInfos['score_ht_away']]
dfGameInfos['score_ft_home'] = [int(number) for number in dfGameInfos['score_ft_home']]
dfGameInfos['score_ft_away'] = [int(number) for number in dfGameInfos['score_ft_away']]

In [ ]:
# *******************************************************
# *******************************************************
PKL_HASH_EMOLEX = "dtr_hash_svn_2016-01-22_08:26:25.pkl"
PKL_REVIEW = 'dtr_hash_svn_2016-01-24_05:14:42.pkl'

# Read SVM(hash emolex) detecter
with open(paths.DETECTER_HOME + PKL_HASH_EMOLEX, 'rb') as f:
    u = pickle._Unpickler(f)
    u.encoding = 'utf-8'
    detector = u.load()

In [ ]:
def AddScorePosNegColumn(df, detecter):
    df = df.copy().reset_index(drop=True)
    score_pos = []
    score_neg = []

    for text in df.text:
        neg, pos = detector.predict_proba([text])[0]
        score_pos.append(pos)
        score_neg.append(neg)

    df['score_pos'] = score_pos
    df['score_neg'] = score_neg

    return df

In [ ]:
# Count Home, Away Emolex
def TweetPNscore(week, team_home, team_away):
    # week = 22
    # team_home = 'Swansea'
    # team_away = 'Watford'

    # Read Single as DF
    dfGame = useful_methods.SingleGameDf(week, team_home, team_away, filtering=True, retweet=RETWEET_STATUS)
    if dfGame is None:
        print(None, None)
        return None

    dfGame.ith_minute = [int(ith_minute) for ith_minute in list(dfGame.ith_minute)]

    # Count Emolex Words
    dfHome = AddScorePosNegColumn(dfGame[(dfGame.side == 'home') & (dfGame.ith_minute <= TIME_LIMIT)], detector)
    dfAway = AddScorePosNegColumn(dfGame[(dfGame.side == 'away') & (dfGame.ith_minute <= TIME_LIMIT)], detector)

    return (
        dfHome.score_pos.sum(), dfHome.score_neg.sum(),
        dfAway.score_pos.sum(), dfAway.score_neg.sum())

In [ ]:
taken_time = time()

counted_game_emolex = []

for week in set(dfGameInfos.GW):
# for week in set([22]):
    for index in range((week - 1) * 10, week * 10):
        # Team names
        team_home = dfGameInfos[dfGameInfos.GW == week].loc[index]['home_team']
        team_away = dfGameInfos[dfGameInfos.GW == week].loc[index]['away_team']

        print("\n\n------------------")
        print(week, team_home, team_away)
        pn_score = TweetPNscore(week, team_home, team_away)
        counted_game_emolex.append(pn_score)
        print(pn_score)

print("[Done:] %.2f" % (time() - taken_time))

In [ ]:
dfGameInfos.loc[220]

In [ ]:
for ith_row in range(len(dfGameInfos)):
    # Team names
    week = dfGameInfos.iloc[ith_row]['GW']
    team_home = dfGameInfos.iloc[ith_row]['home_team']
    team_away = dfGameInfos.iloc[ith_row]['away_team']
    
    print("\n\n[Done]------------------\n")
    print(week, team_home, team_away)

In [ ]:
pn_home_pos = []
pn_home_neg = []
pn_away_pos = []
pn_away_neg = []

for pn_scores in counted_game_emolex:
    if pn_scores is None:
        pn_home_pos.append(-1)
        pn_home_neg.append(-1)
        pn_away_pos.append(-1)
        pn_away_neg.append(-1)
    else:
        pn_home_pos.append(pn_scores[0])
        pn_home_neg.append(pn_scores[1])
        pn_away_pos.append(pn_scores[2])
        pn_away_neg.append(pn_scores[3])


dfGameInfos['pn_home_pos'] = pn_home_pos
dfGameInfos['pn_home_neg'] = pn_home_neg
dfGameInfos['pn_away_pos'] = pn_away_pos
dfGameInfos['pn_away_neg'] = pn_away_neg

dfGameInfos

In [ ]:
# Save as CSV
useful_methods.DFtoCSV(dfGameInfos, paths.DATA_HOME + "EPL/", 'GW23_hash_emolex_pn_score', index=False)
print("[Saved in]: %s" % (paths.DATA_HOME + "EPL/" + 'GW23_hash_emolex_pn_score.csv'))